# Working code


In [20]:
import json
import os
import pandas as pd

# Function to extract table and column pairs
def extract_table_column_pairs(data, parent_key="", result=[]):
    if isinstance(data, dict):
        for key, value in data.items():
            new_key = f"{parent_key}.{key}" if parent_key else key
            if key == 'name':
                table_name = value
                extract_columns(data.get('columns', []), table_name, result)
            extract_table_column_pairs(value, new_key, result)
    elif isinstance(data, list):
        for index, item in enumerate(data):
            new_key = f"{parent_key}[{index}]"
            extract_table_column_pairs(item, new_key, result)

# Function to extract columns within a table
def extract_columns(columns, table_name, result):
    for column in columns:
        column_name = column.get('name')
        column_description = column.get('description', 'N/A')
        source_column = column.get('sourceColumn', 'N/A')
        column_expression = get_column_expression(column)
        if column_name:
            result.append({'Table': table_name, 'Column': column_name, 'SourceColumn': source_column, 'Expression': column_expression, 'Description': column_description})

# Function to get column expression
def get_column_expression(column):
    if 'expression' in column:
        return column['expression']
    else:
        return ''

# Function to extract measures and expressions
def extract_measures_and_expressions(data, parent_key="", result=[]):
    if isinstance(data, dict):
        for key, value in data.items():
            new_key = f"{parent_key}.{key}" if parent_key else key
            if key == 'name':
                table_name = value
                measures = data.get('measures', [])
                extract_measures(measures, table_name, result)
            extract_measures_and_expressions(value, new_key, result)
    elif isinstance(data, list):
        for index, item in enumerate(data):
            new_key = f"{parent_key}[{index}]"
            extract_measures_and_expressions(item, new_key, result)

# Function to extract measures within a table
def extract_measures(measures, table_name, result):
    for measure in measures:
        #print(measure)
        measure_name = measure.get('name')
        measure_description = measure.get('description', 'N/A')
        measure_expression = measure.get('expression', '')
        if measure_name:
            result.append({'Table': table_name, 'Measure': measure_name, 'Expression': measure_expression, 'Description': measure_description})
# Function to extract table partitions
def extract_table_partitions(data, parent_key="", result=[]):
    if isinstance(data, dict):
        for key, value in data.items():
            new_key = f"{parent_key}.{key}" if parent_key else key
            if key == 'name':
                table_name = value
                partitions = data.get('partitions', [])
                extract_partitions(partitions, table_name, result)
            extract_table_partitions(value, new_key, result)
    elif isinstance(data, list):
        for index, item in enumerate(data):
            new_key = f"{parent_key}[{index}]"
            extract_table_partitions(item, new_key, result)

# Function to extract partitions within a table
def extract_partitions(partitions, table_name, result):
    for partition in partitions:
        partition_name = partition.get('name')
        partition_type = partition.get('source', {}).get('type', 'N/A')
        partition_query = partition.get('source', {}).get('query', 'N/A')
        partition_data_source = partition.get('source', {}).get('dataSource', 'N/A')
        if partition_name:
            result.append({'Table': table_name, 'Partition': partition_name, 'Type': partition_type, 'Query': partition_query, 'DataSource': partition_data_source})



# Folder containing JSON files
json_folder = r"C:\Users\sunny.chandel\Desktop\7Eleven-reporting-team\cubes_json"

# Output Excel file
output_excel_file = r"C:\Users\sunny.chandel\Desktop\output_1aug.xlsx"

# Create empty lists to accumulate data
measures_data = []
table_column_data = []
table_partition_data = []

# Loop through JSON files and accumulate data
for file_name in os.listdir(json_folder):
    if file_name.endswith(".json"):
        file_path = os.path.join(json_folder, file_name)
        cube_name = os.path.splitext(file_name)[0]
        
        with open(file_path, 'r', encoding='utf-8') as json_file:
            json_data = json.load(json_file)

        # Extract measures and expressions
        measures_result = []
        extract_measures_and_expressions(json_data, result=measures_result)
        for measure_data in measures_result:
            measure_data['CubeName'] = cube_name
        measures_data.extend(measures_result)

        # Extract table and column pairs
        table_column_result = []
        extract_table_column_pairs(json_data, result=table_column_result)
        for table_column in table_column_result:
            table_column['CubeName'] = cube_name
        table_column_data.extend(table_column_result)

        # Extract table partitions
        table_partition_result = []
        extract_table_partitions(json_data, result=table_partition_result)
        for table_partition in table_partition_result:
            table_partition['CubeName'] = cube_name
        table_partition_data.extend(table_partition_result)

# Create DataFrames with CubeName column at the beginning
df_measures = pd.DataFrame(measures_data)
df_measures = pd.concat([df_measures['CubeName'], df_measures.drop('CubeName', axis=1)], axis=1)

df_table_column = pd.DataFrame(table_column_data)
df_table_column = pd.concat([df_table_column['CubeName'], df_table_column.drop('CubeName', axis=1)], axis=1)

df_table_partition = pd.DataFrame(table_partition_data)
df_table_partition = pd.concat([df_table_partition['CubeName'], df_table_partition.drop('CubeName', axis=1)], axis=1)

# Create Excel file and write data
with pd.ExcelWriter(output_excel_file) as writer:
    df_measures.to_excel(writer, sheet_name='Measures', index=False)
    df_table_column.to_excel(writer, sheet_name='TableColumnPairs', index=False)
    df_table_partition.to_excel(writer, sheet_name='TablePartitions', index=False)

print("DataFrames saved to Excel file:", output_excel_file)


DataFrames saved to Excel file: C:\Users\sunny.chandel\Desktop\output_1aug.xlsx
